In [1]:
from json import JSONDecodeError
import httpx
from pydantic import ValidationError
from sierra_ils_utils import SierraAPIResponse
from sierra_ils_utils.decorators import hybrid_retry_decorator, authenticate

import logging

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.DEBUG)

In [3]:
from sierra_ils_utils import SierraAPI, Models
import json

with open('../.config.json') as f:
  config = json.load(f)

sierra_api = SierraAPI(
    sierra_api_base_url=config.get('sierra_api_base_url'),
    sierra_api_key=config.get('sierra_api_key'),
    sierra_api_secret=config.get('sierra_api_secret')
)

DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/home/ray/Documents/sierra-ils-utils/venv/lib/python3.10/site-packages/certifi/cacert.pem'


In [5]:
result = sierra_api.get(
    'info/token'
)

result

DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.complete
DEBUG:httpcore.connection:connect_tcp.started host='classic.cincinnatilibrary.org' port=443 local_address=None timeout=5.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x7fd879c7ae30>
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x7fd87821d340> server_hostname='classic.cincinnatilibrary.org' timeout=5.0
DEBUG:httpcore.connection:start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x7fd857e4ef80>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'GET']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'GET']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'GET']>
DEBUG:httpcore.http11:receive_response_h

{
    "raw_response": "<Response [200 200]>",
    "response_model_name": "TokenInfo",
    "data": {
        "patronId": null,
        "keyId": "1ebmZG6N2Qw7SklwUBQkxIeTr1pW",
        "grantType": "client_credentials",
        "authorizationScheme": "Bearer",
        "expiresIn": 3515,
        "roles": [
            {
                "name": "Invoices_Write",
                "tokenLifetime": 3600,
                "permissions": [
                    "Invoices_Create",
                    "Invoices_Post",
                    "Invoices_Update"
                ]
            },
            {
                "name": "Fines_Read",
                "tokenLifetime": 3600,
                "permissions": [
                    "Fines_List"
                ]
            },
            {
                "name": "Vendors_Read",
                "tokenLifetime": 3600,
                "permissions": [
                    "Vendors_List"
                ]
            },
            {
                "name"

In [4]:
urls = {
    'get': 'https://httpbin.org/get',
    'get_200': 'https://httpbin.org/status/200',
    'get_500': 'https://httpbin.org/status/500',  # trigger a retry
}

class TestClass:

    @hybrid_retry_decorator(
        # set some testing values in place of defaults
        max_retries=3,  # max_retries=5,
        initial_wait_time=.01,
        initial_exponential_factor=2,
        initial_retries=2,
        fixed_interval=2,  # fixed_interval=150,  # 2.5 minutes
        retry_on_exceptions=None,
        retry_on_status_codes=None,
    )
    def get(self, url):
        with httpx.Client() as client:
            response = client.get(
                url
            )
        
        response_model_name = "SomeModelName"
        httpx_response = response

        try:
            data = response.json()  # This should be structured as per the expected model
        except JSONDecodeError:
            data = {'text': response.text}

        try:
            # Validate and create the SierraAPIResponse instance
            sierra_response = SierraAPIResponse(
                response_model_name=response_model_name,
                data=data,
                raw_response=httpx_response
            )

        except ValidationError as e:
            print("Validation error:", e)

        return sierra_response


# simulate a get request

my_class = TestClass()

for url in urls.keys():
    target_url = urls[url]
    print(target_url)

    result = my_class.get(target_url)
    print(result)

#     with httpx.Client() as client:
#         response = client.get(
#             urls.get(url),
        
#         )

#     # Assuming you have these variables ready
#     response_model_name = "SomeModelName"
#     httpx_response = response

#     try:
#         data = response.json()  # This should be structured as per the expected model
#     except JSONDecodeError:
#         data = {'text': response.text}

#     try:
#         # Validate and create the SierraAPIResponse instance
#         sierra_response = SierraAPIResponse(
#             response_model_name=response_model_name,
#             data=data,
#             raw_response=httpx_response
#         )

#     except ValidationError as e:
#         print("Validation error:", e)

#     print(sierra_response)

DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/home/ray/Documents/sierra-ils-utils/venv/lib/python3.10/site-packages/certifi/cacert.pem'
DEBUG:httpcore.connection:connect_tcp.started host='httpbin.org' port=443 local_address=None timeout=5.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x7fb8c04f2650>
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x7fb8abfaa4c0> server_hostname='httpbin.org' timeout=5.0
DEBUG:httpcore.connection:start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x7fb8c04f2620>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'GET']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'GET']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receiv

https://httpbin.org/get


DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Wed, 06 Dec 2023 04:42:20 GMT'), (b'Content-Type', b'application/json'), (b'Content-Length', b'305'), (b'Connection', b'keep-alive'), (b'Server', b'gunicorn/19.9.0'), (b'Access-Control-Allow-Origin', b'*'), (b'Access-Control-Allow-Credentials', b'true')])
INFO:httpx:HTTP Request: GET https://httpbin.org/get "HTTP/1.1 200 OK"
DEBUG:httpcore.http11:receive_response_body.started request=<Request [b'GET']>
DEBUG:httpcore.http11:receive_response_body.complete
DEBUG:httpcore.http11:response_closed.started
DEBUG:httpcore.http11:response_closed.complete
DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.complete
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/home/ray/Documents/sierra-ils-utils/venv/lib/python3.10/site-packages/certifi/cacert.pem'
DEBUG:httpcore.connection:connect_tcp.started ho

{
    "raw_response": "<Response [200 OK]>",
    "response_model_name": "SomeModelName",
    "data": {
        "args": {},
        "headers": {
            "Accept": "*/*",
            "Accept-Encoding": "gzip, deflate",
            "Host": "httpbin.org",
            "User-Agent": "python-httpx/0.25.2",
            "X-Amzn-Trace-Id": "Root=1-656ffbac-20b12a09370bc3384782eadb"
        },
        "origin": "163.182.20.235",
        "url": "https://httpbin.org/get"
    }
}
https://httpbin.org/status/200
{
    "raw_response": "<Response [200 OK]>",
    "response_model_name": "SomeModelName",
    "data": {
        "text": ""
    }
}
https://httpbin.org/status/500


DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x7fb8c0ee7640>
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x7fb8abfc04c0> server_hostname='httpbin.org' timeout=5.0
DEBUG:httpcore.connection:start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x7fb8c0592da0>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'GET']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'GET']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'GET']>
DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 500, b'INTERNAL SERVER ERROR', [(b'Date', b'Wed, 06 Dec 2023 04:42:20 GMT'), (b'Content-Type', b'text/html; charset=utf-8'), (b'Content-Length', b'0'), (b'Connection', b'keep-alive'), (b'Server', b'g

HTTPStatusError: HTTP 500 Error

{'args': {},
 'headers': {'Accept': '*/*',
  'Accept-Encoding': 'gzip, deflate',
  'Host': 'httpbin.org',
  'User-Agent': 'python-httpx/0.25.2',
  'X-Amzn-Trace-Id': 'Root=1-656febdf-19bd5a43058eafc10cbb2425'},
 'origin': '163.182.20.235',
 'url': 'https://httpbin.org/get'}